# Breast-96-Samples.R as a Notebook 

rMATS 3.2.5 was run on controlled access RNASeq files retrieved experiments stored in the Sequence Read Archive with controlled access managed by dbGaP.   The data were generated under the Gene Tissue Expression.

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## rMATS-final-merged
the rmats-nf NextFlow was executed and the results released here:

## Loading dependencies

In [46]:
library(limma)
library(multtest)
library(Biobase)
library(edgeR)

In [47]:
install.packages('R.utils')
library(R.utils)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [48]:
# jupyter notebooks are in one subdirectory -- data directories are up and over in another - but verify.
getwd()

[1] "/mnt/shared/gcp-user/session_data/sbas/jupyter"

In [49]:
#DS analysis

#inc.iso.counts.mem<-read.csv('/Users/karleg/Downloads/rmats_final.se.jc.ijc.txt',header=TRUE)
inc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.ijc.txt.gz") 

#skip.iso.counts.mem<-read.csv('/Users/karleg/Downloads/rmats_final.se.jc.sjc.txt',header=TRUE)
skip.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.sjc.txt.gz") 

meta.data<-read.csv('../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt',header=TRUE)
head(inc.iso.counts.mem)
head(skip.iso.counts.mem)
head(meta.data)

ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880,SRR1068929,SRR1068953,SRR1068977,SRR1068999,⋯,SRR821573,SRR821581,SRR821602,SRR821626,SRR821653,SRR821690,SRR821715,SRR823967,SRR823991,SRR824015
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,26,247,103,620,494,145,145,139,697,⋯,151,32,62,48,963,25,196,76,72,61
3,1,0,1,0,0,0,1,1,2,⋯,2,1,0,1,3,0,1,0,0,0
4,0,1,1,2,0,0,1,0,2,⋯,0,0,0,0,1,0,0,0,0,0
5,3,0,2,3,6,1,1,1,5,⋯,3,2,0,1,6,0,2,0,0,0
6,2,1,2,5,6,1,1,0,5,⋯,1,1,0,0,4,0,1,0,0,0


ID,SRR1068788,SRR1068808,SRR1068832,SRR1068855,SRR1068880,SRR1068929,SRR1068953,SRR1068977,SRR1068999,⋯,SRR821573,SRR821581,SRR821602,SRR821626,SRR821653,SRR821690,SRR821715,SRR823967,SRR823991,SRR824015
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2,0,1,3,6,1,0,0,3,⋯,1,1,0,0,3,0,1,0,0,0
2,0,0,0,1,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,⋯,0,0,0,0,2,0,1,0,0,0
5,0,5,3,8,4,0,3,0,3,⋯,9,3,1,2,3,0,1,0,1,0
6,11,119,36,284,207,60,63,43,295,⋯,52,13,14,9,338,8,63,25,20,18


,Run,analyte_type,Assay.Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,body_site,⋯,data_type..run.,product_part_number..exp.,product_part_number..run.,sample_barcode..exp.,sample_barcode..run.,is_technical_control,target_set..exp.,primary_disease..exp.,secondary_accessions..run.,Alignment_Provider..run.
,<fct>,<fct>,<fct>,<int>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<lgl>,<fct>,<fct>,<lgl>
1,SRR2911715,RNA,RNA-Seq,150,3852895500,PRJNA244100,SAMN04216864,Cloud Testing,HG00103,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA
2,SRR2911716,RNA,RNA-Seq,150,4885577400,PRJNA244100,SAMN04216866,Cloud Testing,HG00154,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA
3,SRR2911718,RNA,RNA-Seq,150,2690545500,PRJNA244100,SAMN04216863,Cloud Testing,NA18910,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA
4,SRR2911719,RNA,RNA-Seq,150,2699599350,PRJNA244100,SAMN04216865,Cloud Testing,NA19200,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA
5,SRR2911720,RNA,RNA-Seq,152,4300467752,PRJNA244100,SAMN04216865,Cloud Testing,NA19200,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA
6,SRR2911717,RNA,RNA-Seq,150,2666546700,PRJNA244100,SAMN04216863,Cloud Testing,NA18910,Lymphoblastoid cell line,⋯,,,,NA,NA,,NA,,,NA


Keep only the runs that are in the ijc count list (assuming ijc and sjc are the same).  As well, name the rows with the junction id column and then make the matrix just about the counts.

In [50]:
#dimensions before we make the changes.
dim(inc.iso.counts.mem)
dim(skip.iso.counts.mem)
dim(meta.data)

#logic to keep only those runs (rows) that are in the matrix of ijc counts (columns)
keep.meta.data <- meta.data$Run %in% colnames(inc.iso.counts.mem)
table(keep.meta.data)
reduced.meta.data <- meta.data[keep.meta.data==TRUE,]

# there is one more column that has the junction ID, preserve that as the row number
rownames(inc.iso.counts.mem)<-inc.iso.counts.mem$ID
rownames(skip.iso.counts.mem)<-skip.iso.counts.mem$ID

#remove the id and now we should have parity between columns (which are the runs) for ijc, sjc with rows of the metadata (which are also runs)
inc.iso.counts.mem  <-inc.iso.counts.mem[,-1]
skip.iso.counts.mem <-skip.iso.counts.mem[,-1]
dim(inc.iso.counts.mem)
dim(skip.iso.counts.mem)
dim(reduced.meta.data)

[1] 42611  8674

[1] 42611  8674

[1] 9784   79

keep.meta.data
FALSE  TRUE 
 1111  8673 

[1] 42611  8673

[1] 42611  8673

[1] 8673   79

Cool -- now we need to ensure the ordering of the metadata (rownames) is the same as the colnames, ijc and sjc.  Strings as factors, the metadata was read in with StringsAsFactors == TRUE, we will rearrange the order of the columns for our ijc, sjc count data to match that of the Run column of the metadata, our column names in the count data hold the Run names, we need to ensure we compare as.character (colnames can not be factors).

using library tibble,  which we start by converting to a tibble data frame (tbl_df). Tibble is a modern rethinking of data frame providing a nicer printing method. This is useful when working with large data sets.   Using tibble, we can rearrange the columns as the column name.   Method to use is converting the ijc, sjc count data.frame to a tibble data frame.

In [72]:
library(tibble)

In [75]:
meta.data.colnames   <- as.character(reduced.meta.data$Run)
inc.iso.counts.mem2  <- as_tibble(inc.iso.counts.mem)
inc.iso.counts.mem2  <- inc.iso.counts.mem2[,c(meta.data.colnames)]
skip.iso.counts.mem2 <- inc.iso.counts.mem2[,c(meta.data.colnames)]
dim(skip.iso.counts.mem2)
dim(inc.iso.counts.mem2)

[1] 42611  8673

[1] 42611  8673

Remove samples that match '11IL0' from the ijc, sjc and metadata files and only look at breast tissue -- @Christina - we should make this a function that we pass all the tissues - like what I did in one of the figures...

In [79]:
keep.meta.data <- (!grepl('11ILO',reduced.meta.data$Sample.Name) & (reduced.meta.data$body_site %in% 'Breast - Mammary Tissue'))
table(keep.meta.data)
inc.iso.counts.mem2  <-inc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]
skip.iso.counts.mem2 <-skip.iso.counts.mem2[                    ,keep.meta.data==TRUE]
reduced.meta.data    <-reduced.meta.data   [keep.meta.data==TRUE,                    ]
dim(inc.iso.counts.mem2)
dim(skip.iso.counts.mem2)

keep.meta.data
FALSE  TRUE 
 8470   191 

[1] 42611   191

[1] 42611   191

Make a single matrix combining the ijc, sjc counts 

In [80]:
exprs.mat <- matrix(data=cbind(as.matrix(skip.iso.counts.mem2),
                               as.matrix(inc.iso.counts.mem2)),
                    ncol=2*ncol(skip.iso.counts.mem2),
                    nrow=nrow(skip.iso.counts.mem2),
                    dimnames=list(1:nrow(skip.iso.counts.mem2),
                                  c(colnames(skip.iso.counts.mem2),
                                    paste(colnames(skip.iso.counts.mem2),'2',sep=''))))



Now make groups for our analysis focused on sex but with two sub classifications, skip for our sjc counts and inc for our ijc counts - representing the number of reads that have skipped the exon and the number of reads that have included the exon.

In [84]:
keep.events<-rep(T,nrow(exprs.mat))

groups=c(paste0(reduced.meta.data$sex,'-skip'),paste0(reduced.meta.data$sex,'-inc'))


keep only the groups and the rows that have non-zero values

In [85]:
length(groups)
dim(exprs.mat)
groups<-groups[colSums(exprs.mat)>0]
exprs.mat<-exprs.mat[,colSums(exprs.mat)>0]
length(groups)
dim(exprs.mat)

[1] 382

[1] 42611   382

[1] 382

[1] 42611   382

In [ ]:
#zero.rows=rep(F,nrow(exprs.mat))

#for (group in c('male-skip','female-skip','male-inc','female-inc'))
  
#  zero.rows=zero.rows | (rowSums(exprs.mat[,groups %in% group] >0) < (1/4)*min(table(groups)))

#exprs.mat[zero.rows,]=0

#for (group in c('male-skip','female-skip','male-inc','female-inc'))
  
#  keep.events<-keep.events & (rowSums((exprs.mat[,groups %in% group]) >= 1) >= (1/4)*min(table(groups)))

#rm(exprs.mat)


separate the matrices counts into male and female

In [88]:
inc.counts.male   <-inc.iso.counts.mem2 [,reduced.meta.data$sex=='male']

inc.counts.female <-inc.iso.counts.mem2 [,reduced.meta.data$sex=='female']

skip.counts.male  <-skip.iso.counts.mem2[,reduced.meta.data$sex=='male']

skip.counts.female<-skip.iso.counts.mem2[,reduced.meta.data$sex=='female']


make the matrix, combining now the male and female counts - achieving a rearrangement count columns included counts male, female, followed by skiped counts male female

In [89]:
inc.counts.mat<-cbind(inc.counts.male,inc.counts.female)

rownames(inc.counts.mat)<-rownames(inc.iso.counts.mem)

skip.counts.mat<-cbind(skip.counts.male,skip.counts.female)

rownames(skip.counts.mat)<-rownames(skip.iso.counts.mem)

counts.mat<-cbind(inc.counts.male,inc.counts.female,skip.counts.male,skip.counts.female)

rownames(counts.mat)<-rownames(inc.iso.counts.mem)

counts.mat<-counts.mat[keep.events,]

this gives us then the ability to create an isoform.   For our modelling, included counts will be 1, skipped will be 0.   Sex will be modeled as well, this will be a block.

In [90]:
isoform<-c(rep(1,ncol(inc.counts.male)+ncol(inc.counts.female)),rep(0,ncol(skip.counts.male)+ncol(skip.counts.female)))
sex<-c(rep(1,ncol(inc.counts.male)),rep(0,ncol(inc.counts.female)),rep(1,ncol(skip.counts.male)),rep(0,ncol(skip.counts.female)))
block=rep(c(1:ncol(inc.counts.male),(1+ncol(inc.counts.male)):(ncol(inc.counts.female)+ncol(inc.counts.male))),2)

keep only those rows which have non-zero columns

In [91]:
dim(counts.mat)
keep.cols<-colSums(counts.mat)>0
table(keep.cols)
isoform<-isoform[keep.cols]

sex<-sex[keep.cols]

block<-block[keep.cols]

counts.mat<-counts.mat[,keep.cols]
dim(counts.mat)

[1] 42611   382

keep.cols
TRUE 
 382 

[1] 42611   382

finally we are able to perform our analysis.

In [ ]:
norm.factors=calcNormFactors(DGEList(counts=counts.mat[,isoform==1]+counts.mat[,isoform==0]))

counts.mat=DGEList(counts=counts.mat)

counts.mat$samples$norm.factors=rep(norm.factors$samples$norm.factors,2)

design <- model.matrix(~sex+isoform+sex*isoform)

correlation =  duplicateCorrelation(voom(counts.mat)$E,design,block=block)$consensus.correlation

v <- voom(counts.mat, design,block = block, correlation = correlation)

fit <- lmFit(v, design,block = block, correlation = correlation)

fit <- eBayes(fit, robust=TRUE)

res=topTable(fit, coef='sex:isoform',number=nrow(counts.mat))

write.table(res[res$adj.P.Val<=0.05 & abs(res$logFC)>=log2(1.5),],paste(paste('Breast - Mammary Tissue',collapse='.'),'se.txt',sep=''),row.names = T,col.names = T,quote = F)

write.table(rownames(res),paste0(paste('Breast - Mammary Tissue',collapse='.'),'all_genes.txt'))


In [ ]:
#Run Ontologizer

meta.data<-read.table('/Users/karleg/Downloads/fromGTF.SE.txt',sep='\t',header=TRUE)

de.tab<-read.table('Breast - Mammary Tissuese.txt')

de.tab.with.meta<-merge(de.tab,meta.data,by.x='row.names',by.y='ID')

all.genes<-read.table('Breast - Mammary Tissueall_genes.txt')

write.table(meta.data$geneSymbol[meta.data$ID %in% all.genes[,1]],'universe.txt',quote = F,row.names = F,col.names = F)

write.table(de.tab.with.meta$geneSymbol,'gene_set.txt',quote = F,row.names = F,col.names = F)

system('java -jar /Users/karleg/Ontologizer/Ontologizer.jar -g /Users/karleg/Ontologizer/go.obo -a /Users/karleg/Ontologizer/goa_human.gaf -s gene_set.txt -p universe.txt -c Term-For-Term -m Benjamini-Hochberg -n')


# Retrieving the GTEx archive

We used the R package [{yarn}](https://bioconductor.org/packages/release/bioc/html/yarn.html) to retrieve the GTEx Biobank data. In order to downloaded the latest GTEx version 8.0 for RNA-seq and genotype data (phs000424.v8.v2), released 2019-08-26, we created a fork of the package's GitHub repository and created a new version of the function **`yarn::downloadGTEx()`**, namely **`yarn::downloadGTExV8()`** to download this release. 

We used the function to perform quality control, gene filtering and normalization pre-processing on the GTEx RNA-seq data, as described in (Paulson et al, 2017). This pipeline tested for sample sex-misidentification, merged related sub-tissues and performed tissue-aware normalization using the **`{yarn::qsmooth}`**  function (Hicks et al, 2017).

We have archived the output of the **`yarn::downloadGTExV8()`** function, which is an `ExpressionSet` object in the repo `lifebitai/lifebitCloudOSDREgtex` for replicability and decreasing the runtime of this analysis. Below we retrieve this `gtex.rds` object from the GitHub releases using the **`{ropensci/piggyback}`** package, but we have also added the relevant command to retrieve the data from GTEx and generate the `ExpressionSet` object using  **`yarn::downloadGTExV8()`**. For the current analysis we are utilising a compute resource with 8 vCPUs and 60 GB of memory available.



In [ ]:
# Load with readRDS() if gtex.rds available in data/
if ("gtex.rds" %in% list.files("../data/")) {
    message("Loading GTEx v8 rds object with readRDS from ../data/gtex.rds ..\n")   
    obj <- readRDS(file = "../data/gtex.rds")
    message("Done!\n")
    message("Generating sha256sum for gtex.rds ..\n")    
    message(system("sha256sum ../data/gtex.rds", intern = TRUE))
    message("Done!\n")
}

# Download from archive if not available in ../data
if (!("gtex.rds" %in% list.files("../data/"))) {
    message("Downloading GTEx v8 from GitHub Releases archive into the ../data/ directory ..\n")
    piggyback::pb_download(file = "gtex.rds", 
                                repo = "lifebit-ai/lifebitCloudOSDREgtex", 
                                tag  = "fig1c_archive", 
                                dest = "../data/")
    message("Generating sha256sum for gtex.rds ..")    
    message(system("sha256sum ../data/gtex.rds", intern = TRUE))
    message("Done!\n")    
    message("Loading GTEx v8 rds object with readRDS from ../data/gtex.rds ..\n")    
    obj <- readRDS(file = "../data/gtex.rds" )
    message("Done!\n")
}

# Download with yarn if you wish, this requires several minutes to complete
if (!("gtex.rds" %in% list.files("../data/"))) {
    message("Downloading GTEx v8 with 'yarn::downloadGTExV8()'")
    obj <- yarn::downloadGTExV8(type='genes',file='../data/gtex.rds')
    message("Done!")

}

In [ ]:
# Confirm that it is an expression set.
# and check the dimensions of the objects, and the phenotype information of the objects
class(obj) 
dim(phenoData(obj))
dim(obj)

### NOTE: 

We observe above that our phenotype data have 2 more observations than our expression data,  let's inspect what are these samples:

In [ ]:
sample_names=as.vector(as.character(colnames(exprs(obj))))
length(sample_names)

pheno_sample_names=as.vector(as.character(rownames(pData(obj))))
length(pheno_sample_names)

if (length(pheno_sample_names) > length(sample_names)) {
    superset <- pheno_sample_names
    subset   <- sample_names    
} 

if (length(pheno_sample_names) < length(sample_names)) {
    superset <- sample_names
    subset   <- pheno_sample_names   
} 

non_overlaps <- setdiff( superset, subset)

message("The non-overlapping IDs between pheno and count data are:\n\n", 
        paste(non_overlaps, collapse = "\n") )

Let's keep only the overlapping IDs:

In [ ]:
logical_match_names=pheno_sample_names %in% sample_names
length(logical_match_names)
table(logical_match_names)
pData(obj) <- (pData(obj)[logical_match_names==TRUE,])
dim(pData(obj))
dim(obj)

In [ ]:
install.packages("R.utils")

In [ ]:
library(R.utils)

We have currently all of our runs from rMATS stored with the SRR number - this maps to the annotation via the SraRunTable$biospecimen_repository_sample_id. 

What we will do here for scalling is read all the inc for this step, and then we will add a separate step for ijc/sjc analysis for comparison purposes.

This step assumes that you either mounted the data from the bucket or you have used (after installing, the gsutil tools from google (on a google platform) or aws cli tools (on an amazon platform).  Or the equivalent on an Azure platform

In [ ]:
se.jc.inc <- data.table::fread("../data/rmats_final.se.jc.inc.txt.gz") 

In [ ]:
se.jcec.inc <- data.table::fread("../data/rmats_final.se.jcec.inc.txt.gz") 

In [ ]:
ri.jc.inc <- data.table::fread("../data/rmats_final.ri.jc.inc.txt.gz")

In [ ]:
ri.jcec.inc <- data.table::fread("../data/rmats_final.ri.jcec.inc.txt.gz")

In [ ]:
mxe.jc.inc <- data.table::fread("../data/rmats_final.mxe.jc.inc.txt.gz")

In [ ]:
mxe.jcec.inc <- data.table::fread("../data/rmats_final.mxe.jcec.inc.txt.gz")

In [ ]:
a3ss.jc.inc <- data.table::fread("../data/rmats_final.a3ss.jc.inc.txt.gz")

In [ ]:
a3ss.jcec.inc <- data.table::fread("../data/rmats_final.a3ss.jc.inc.txt.gz")

In [ ]:
dim(se.jc.inc)

In [ ]:
head (se.jc.inc)

#  Now we want to replace all *dashes* with **dots "."**

In [ ]:
pData(obj)$SAMPID[1]
pData(obj)$SAMPID                            <- gsub('-','\\.',pData(obj)$SAMPID)
pData(obj)$SAMPID[1]

In [ ]:
colnames(pData(obj))

In [ ]:
head(exprs(obj))

This data file needs to be acquired from a release. This is done nicely using the piggyback tool to download from the stored release.  To run this command to access a private release, a user needs to generate their own github token and then in an R or Jupyter Console window use the Sys.setenv(GITHUB_TOKEN="xxxx") command to set it.

In [ ]:
# devtools::install_github("ropensci/piggyback@87f71e8", upgrade="never")
piggyback::pb_download(
repo = "TheJacksonLaboratory/sbas", 
file = "SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz", 
tag  = "GTExV8.v1.0", 
dest = "../data/")

In [ ]:
# From SraRunTable
meta.data           <-  data.table::fread("../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz")

# Reform GTEX ids in meta.data$biospecimen_repository_sample_id to match pData$SAMPID
meta.data$SAMPID<- gsub("-","." , meta.data$biospecimen_repository_sample_id)

# Merge the 2 metadata tables by column SAMPID
merged <- merge(meta.data, pData(obj), by = "SAMPID")
dim(merged)

# something odd about the DTHHRDY column, rendered effective through as.numeric.
# of note, the cause because cause + 1, so that non-intubation death case in original is 0,
# becomes 1 in the death_cause

table(pData(obj)$DTHHRDY)
length(pData(obj)$DTHHRDY)
is.numeric(pData(obj)$DTHHRDY)
death_cause <- as.numeric(pData(obj)$DTHHRDY)
table(death_cause)
non_intubation_death_cause <- death_cause == 1
table(non_intubation_death_cause)
lung_non_intubation <- (pData(obj)$SMTSD == "Lung")[non_intubation_death_cause == TRUE]
table(lung_non_intubation)

table(merged$DTHHRDY)
merged$death_cause = as.numeric(merged$DTHHRDY)
table(merged$death_cause)

# Write dataframe into .csv file
FILE = paste0('../data/merged_metadata', '.csv')
write.table( merged,
             file      = FILE,
             append    = FALSE,
             quote     = FALSE,
             sep       = ",",
             row.names = F,
             col.names = T)

non_intubation_death_cause <- merged$death_cause == 1
table(non_intubation_death_cause)

lung_non_intubation  <- merged[(merged$SMTSD == "Lung" & non_intubation_death_cause == TRUE), ]
dim(lung_non_intubation)

breast               <- merged[merged$SMTSD == "Breast - Mammary Tissue"  , ]
dim(breast)


In [ ]:
colnames(merged)

In [ ]:
dim(se.jc.inc)
ID <- se.jc.inc [,1]
se.jc.inc <- data.matrix(se.jc.inc[,-1])
dim(se.jc.inc)

In [ ]:
head(colnames(se.jc.inc))

We have a few more samples with metadata than what we have in our matrix (8781 vs 8673), so limit the metadata only to those within the rmats sample

In [ ]:
rmats_names=as.vector(as.character(colnames(se.jc.inc)))
length(rmats_names)
rmats_names[1:10]
merged_names=as.vector(as.character(merged$Run))
length(merged_names)
merged_names[1:10]
if (length(merged_names) > length(rmats_names)) {
    superset <- merged_names
    subset   <- rmats_names    
} 

if (length(merged_names) < length(rmats_names)) {
    superset <- rmats_names
    subset   <- merged_names   
} 

#non_overlaps <- setdiff( superset, subset)
rmats_merged_intersect <- intersect(superset, subset)
length(rmats_merged_intersect)
#length(non_overlaps)
#message("The non-overlapping IDs between merged and rmats count data are:\n\n", 
#        paste(non_overlaps, collapse = "\n") )

In [ ]:
length(rmats_merged_intersect)
length(rmats_names)
length(merged_names)

keep_merged <- merged_names %in% rmats_merged_intersect
keep_rmats  <- rmats_names  %in% rmats_merged_intersect

table(keep_merged)
table(keep_rmats)

In [ ]:
head(keep_merged)
head(keep_rmats)

In [ ]:
dim(merged)
dim(se.jc.inc)

Now we have our truth statements, and we have at a counts table (`merged_se.jc.inc` as a datamatrix) that can be reconciled with the metadata, `merged_rmats`

In [ ]:
merged_rmats     <- merged   [keep_merged==TRUE,]
merged_se.jc.inc <- se.jc.inc[                 ,keep_rmats==TRUE]
dim(merged_rmats)
dim(merged_se.jc.inc)


#  Differential Expression using `{edgeR}`

For gene expression analysis, we used the exprs(obj) function to obtain the counts matrix via transcript_counts_matrix <- exprs(obj).  We have our metadata in the 'merged' file this file is a different size at the moment than our sample file.  That is we have more metadata then we

To use the **`edgeR::DGEList()`** function, we need to transpose our `transcript_counts_matrix` so that the length of group is equal to the number of columns in our counts (`transcript_counts_matrix`). You will get an error from the   **`edgeR::DGEList()`** function (counts = x, group = group) if the length of group is not equal to the number of columns in counts.

In [ ]:
groupiso <- factor(merged_rmats$SEX)

In [ ]:
table(groupiso)

In [ ]:
length(groupiso)
dim(merged_se.jc.inc)

In [ ]:
if (!("DGENormFactors.y.se.jc.inc.rds" %in% list.files("../data"))) {
    y.se.jc.inc <- edgeR::DGEList(counts = merged_se.jc.inc, 
                                  group  = groupiso)
    message("Done!\n")    
    message("Calculating normalization factors on `DGEList` object to scale the raw library sizes with 'edgeR::DGEList()' ..")    
    y.se.jc.inc <- edgeR::calcNormFactors(y.se.jc.inc)
    message("Done!\n")    
    message("Saving normalization factors object in '../data/DGENormFactors.y.se.jc.inc.rds' ..")        
    saveRDS(y.se.jc.inc, file = "../data/DGENormFactors.y.se.jc.inc.rds")
    message("Done!\n")    
    message("Generating sha256sum for DGENormFactors.y.se.jc.inc.rds ..\n")    
    message(system("sha256sum ../data/DGENormFactors.y.se.jc.inc.rds", intern = TRUE))
    message("Done!\n")
}

attributes(y.se.jc.inc)

In [ ]:
dim(y.se.jc.inc)

The challenge with the `inc` number is that it is a calculated `percent spliced in or psi` number.  For the statistics, it is best to have counts - which is where the `ijc` and `sjc` numbers come in and can be used for the calculations.  Using the `psi` we should calculate by, say `1000` to do differental analysis on expected count data and treat it as an integer

In [ ]:
y.se.jc.inc_as_counts = y.se.jc.inc

In [ ]:
y.se.jc.inc_as_counts$counts <- y.se.jc.inc$counts * 1000

In [ ]:
mode(y.se.jc.inc_as_counts$counts) <- "integer"

In [ ]:
dim(y.se.jc.inc_as_counts$counts)

In [ ]:
# For Guy -- does this do what you are expecting -- I am confused because what you get when you
#        ask for the min (table(groups)) is the smaller sized group -- which in this case is 
#        female -- it will help the reader to know what you are doing here with the statement.
#        one can read what it is doing but not understand your objective.groups <- pData(obj)$SEX
# keep.events <- rep(TRUE, nrow(y))
# for (group in c(1,2)) {
#    keep.events <- keep.events & 
#                   rowSums(cpm(y[,groups %in% group]) > 1) >= 0.25*min(table(groups))
# }

#  From Anne - I believe the objective in this step is to keep only those genes that are in the
#    that are above the threshold of expression for the lower quartile of all sex specific genes.
#    groups = (1,2) -- lots of confusion in logic between groups and group and male and female
#    I recommend we use male and female.
#    two errors then in the above loop - 
#     1. min(table(groups)) will always return the
#        length of the number of samples that are female 
#        (which is 5978 for this v8 GTEx)
#     2. sum(table(group)) will return the number of samples that are either male or female
#        depending upon whether you are in the loop for male or female consideration.

#    If my assumption is true - I recommend replacing it with the following.

#  this should be replaced 


In [ ]:
keep.events <- rep(TRUE, nrow(y.se.jc.inc_as_counts))
nrow(y.se.jc.inc_as_counts)

#
keep.events  <- keep.events & rowSums(edgeR::cpm(y.se.jc.inc_as_counts$counts[,groupiso == 1]) > 1) >= 0.25*length(groupiso==1)
#
# now keep all the male subsets or the female subsets meeting our criteria
#
keep.events2 <- keep.events | rowSums(edgeR::cpm(y.se.jc.inc_as_counts$counts[,groupiso == 2]) > 1) >= 0.25*length(groupiso==2)
table(keep.events)
table(keep.events2)

In [ ]:
# Load with readRDS() if `../data/reduced_y.se.jc.inc_as_counts.rds` available in data/
#if  ('reduced_y.se.jc.inc_as_counts.rds' %in% list.files("../data/")){
#    message("Loading  `reduced_y.rds` and  `reduced_obj` objects with `readRDS()` from ../data/ \n")    
#    reduced_reduced_y.se.jc.inc_as_counts   <- readRDS(file = "../data/reduced_reduced_y.se.jc.inc_as_counts.rds")
#    message("Done!")    
#}


# Create if not available in ../data
#if  (!('reduced_y.se.jc.inc_as_counts.rds' %in% list.files("../data/"))) {

    message("Creating `reduced_y.se.jc.inc_as_counts.rds` keeping only rows that match `keep.events2` list ..")
    reduced_y.se.jc.inc_as_counts<- y.se.jc.inc_as_counts[keep.events2,]
    message("Done!\n")    
       
    message("Saving `reduced_y.se.jc.inc_as_counts.rds` in ../data/")        
    saveRDS(reduced_y.se.jc.inc_as_counts, file = "../data/reduced_y.se.jc.inc_as_counts.rds")
    message("Done!\n")     
#}

dim(reduced_y.se.jc.inc_as_counts)

# Separate the analysis by male and by female

In [ ]:
dim(reduced_y.se.jc.inc_as_counts)
dim(merged_rmats)

In [ ]:
reorder_rmats_idx = match(colnames(reduced_y.se.jc.inc_as_counts), merged_rmats$Run)
ordered_merged_rmats = merged_rmats[reorder_rmats_idx,]

In [ ]:
male   <- ordered_merged_rmats$SEX==1
female <- ordered_merged_rmats$SEX==2
table(ordered_merged_rmats$SEX)
table(male)
table(female)
ordered_merged_rmats_male   <- ordered_merged_rmats[male==TRUE,]
ordered_merged_rmats_female <- ordered_merged_rmats[female==TRUE,]
dim(ordered_merged_rmats_male)

In [ ]:
# changing from SMTS to SMTSD - more granularity -- but we could produce both figures
tissue_groupiso        <- factor(ordered_merged_rmats$SMTSD)
tissue_groupiso_male   <- factor(ordered_merged_rmats_male$SMTSD)
tissue_groupiso_female <- factor(ordered_merged_rmats_female$SMTSD)

In [ ]:
# good sanity check, the male set does not have any vaginas or uterus
table (tissue_groupiso_male)

In [ ]:
# and the females have no prostate or testis
table(tissue_groupiso_female)

Reproducing previous results using the `yarn` expression object, loop through the tissues and for those tissues that are shared between the two sexes perform a differential gene analysis on a per tissue basis.


In [ ]:
tissue_male_female <- tissue_groupiso_male %in% tissue_groupiso_female
table(tissue_male_female)

In [ ]:
tissue_shared_male_female <- factor(tissue_groupiso_male[tissue_male_female==TRUE])
table(tissue_shared_male_female)

In [ ]:
# SEX is coded 1 == Male
#              2 == Female
sex = factor(ordered_merged_rmats$SEX)

Let's now define a function named `fit_tissue()`that accepts two arguments, the `tissue` and an `object` and create the **model matrix** based  that tissue's sex. We will perform a linear fit after calculating normal factors (based on the library size) and calculate the dispersion using `{voom}` (mean variance model of dispersion). We are saving the resulting matrixes as files.


## rMATS RNASeq-MATS.py produces 10 different output types which get assembled into as type junction ID by sample ID matrices

### Alternative Splice Site Types are: (se, a3ss, a5ss, mxe, ri)

 This is input as ARGV1 into variable 'astype'

  * Skipped Exon events (se),
  * Alternative 3' splice site (a3ss),
  * Alternative 5' splice site (a5ss),
  * Mutually exclusive exon (mxe),
  * and retention intron (ri)

### There are two different kinds of junction counts

  * jc = junction counts - reads that cross the junction
  * jcec = junction counts plus reads on the target (such as included exon

### And the count type -- there are 5 types

  * inclusion levels (percent spliced in)
  * included junction counts (ijc)
  * skipped junction counts (sjc)
  * inclusion length (inclen)
  * skipped length (skiplen)

### function: fit_iso_tissue 

fit_iso_tissue expects the following input:

  * the tissue of interest (SMSTD) 
  * an ordered_merged_rmats -- which will be ordered to fit the count matrix
  * count matrix (inc or ijc & sjc merged)
  * splice type (a3ss, a5ss, mxe, ri or se)
  * junction_count type (jc or jcec)
  * count type (inc or the merged ijc,sjc)
  
### reordering to match annotations between count matrix and annotation matrix

Common problem is to match specifically the rows of an annotation matrix with the columns of a count matrix
`match` is the function that gives the re-ordering index required to accomplish this

  

In [ ]:
fit_iso_tissue <- function (tissue, merged_rmats, counts, pVal, logFC, splice_type, junction_count, count_type) {
    tissue_true    <- merged_rmats$SMTSD == tissue
    tissue_obj     <- merged_rmats[tissue_true ==TRUE,]
    tissue_counts  <- counts[, tissue_true == TRUE]
    tissue_sex     <- factor(tissue_obj$SEX)
    tissue_design  <- model.matrix(~tissue_sex)
    y_tissue       <- DGEList(counts=tissue_counts, group=tissue_sex)
    y_tissue       <- calcNormFactors(y_tissue)
    y_tissue_voom  <- voom(y_tissue, tissue_design)
    fit_tissue     <- lmFit(y_tissue_voom, tissue_design)
    fit_tissue     <- eBayes(fit_tissue, robust=TRUE)
    results_tissue <- topTable (fit_tissue, coef='tissue_sex2', number=nrow(y_tissue))

    refined_filename = paste(paste(paste(paste(paste(paste(paste("../data",gsub(" ","",tissue), sep="/"),
                                                                        splice_type,sep="_"),
                                                                     junction_count,sep="_"),
                                                                         count_type,sep="_"),
                                                                               pVal,sep="_"),
                                                                              logFC,sep="_"),
                                                                          "DGE.csv",sep="_")

    filename         = paste(paste(paste(paste(paste("../data",gsub(" ","",tissue), sep="/"),
                                                                     splice_type   ,sep="_"),
                                                                     junction_count,sep="_"),
                                                                         count_type,sep="_"),
                                                                          "DGE.csv",sep="_")
    
    refined_filenameGenes = paste(paste(paste(paste(paste(paste(paste("../data",gsub(" ","",tissue), sep="/"),
                                                                        splice_type,sep="_"),
                                                                     junction_count,sep="_"),
                                                                         count_type,sep="_"),
                                                                               pVal,sep="_"),
                                                                              logFC,sep="_"),
                                                                    "all_genes.csv",sep="_")
 
    refined_results_tissue <- results_tissue[results_tissue$adj.P.Val<=pVal & abs(results_tissue$logFC)>=log2(logFC),]

    write.table(results_tissue,                           filename,      sep=',', quote=FALSE)
    write.table(refined_results_tissue,           refined_filename,      sep=',', quote=FALSE)
    write.table(rownames(refined_results_tissue), refined_filenameGenes, sep=',', quote=FALSE)
    
    return (results_tissue)
}

In [ ]:
tissues <- levels(tissue_shared_male_female)
tissues
length(tissues)

In [ ]:
# debugging the function with using the 'Thyroid' tissue
#thyroid_logFC <- fit_iso_tissue('Thyroid', ordered_merged_rmats,reduced_y.se.jc.inc_as_counts, "se", "jc","inc_as_counts")
#thyroid_logFC
# 
all_se_jc_inc_logFC <- lapply(X              = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.se.jc.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "se",
                              junction_count = "jc",
                              count_type     = "inc_as_counts")

In [ ]:
all_se_jcec_inc_logFC<- lapply(X             = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.se.jcec.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "se",
                              junction_count = "jcec",
                              count_type     = "inc_as_counts")

In [ ]:
all_ri_jc_inc_logFC <- lapply(X              = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.ri.jc.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "ri",
                              junction_count = "jc",
                              count_type     = "inc_as_counts")


In [ ]:
all_ri_jcec_inc_logFC<- lapply(X             = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.ri.jcec.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "ri",
                              junction_count = "jcec",
                              count_type     = "inc_as_counts")

In [ ]:
all_mxe_jc_inc_logFC <- lapply(X             = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.mxe.jc.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "mxe",
                              junction_count = "jc",
                              count_type     = "inc_as_counts")


In [ ]:
all_mxe_jcec_inc_logFC<- lapply(X            = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.mxe.jcec.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "mxe",
                              junction_count = "jcec",
                              count_type     = "inc_as_counts")

In [ ]:

all_a3ss_jc_inc_logFC  <- lapply(X           = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.a3ss.jc.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "a3ss",
                              junction_count = "jc",
                              count_type     = "inc_as_counts")

In [ ]:
all_a3ss_jcec_inc_logFC<- lapply(X           = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.a3ss.jcec.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "a3ss",
                              junction_count = "jcec",
                              count_type     = "inc_as_counts")

In [ ]:
all_a5ss_jc_inc_logFC  <- lapply(X           = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.a5ss.jc.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "a5ss",
                              junction_count = "jc",
                              count_type     = "inc_as_counts")

In [ ]:
all_a5ss_jcec_inc_logFC<- lapply(X           = tissues, 
                              FUN            = fit_iso_tissue,
                              merged_rmats   = ordered_merged_rmats,
                              counts         = reduced_y.a5ss.jcec.inc_as_counts,
                              pVal           = 0.05,
                              logFC          = 1.5,
                              splice_type    = "a5ss",
                              junction_count = "jcec",
                              count_type     = "inc_as_counts")

In [ ]:
length(all_se_jc_inc_logFC)

In [ ]:
filenames <- list.files("../data", pattern="*_as_counts_DGE.csv", all.files=FALSE, full.names=TRUE)
length(filenames)
# preserve the ordered rownames for later assignment to matrix
fullfilename <-filenames[1]
all_se_jc_inc_logFC_mat    <- read.csv(fullfilename)
all_se_jc_inc_logFC_mat    <- all_se_jc_inc_logFC_mat[order(rownames(all_se_jc_inc_logFC_mat)),]
all_se_jc_inc_logFC_mat_rownames <- as.character(rownames(all_se_jc_inc_logFC_mat)) 
head(all_se_jc_inc_logFC_mat_rownames)

## Make a matrix for a heat map of the more stringent cutoff values
We have the set of files with more stringent cutoffs (pVal = 0.05 and absolute log Fold Change > 1.5).  However, the most significant expression values will be different depending upon the tissue, we need to build the matrix populating with values for genes already seen (as encoded by the first column) as well as for those not seen.

In [ ]:
# The easiest way to get dplyr is to install the whole tidyverse:
require(dplyr)
remove.packages(c("plyr"))

In [ ]:
#remove.packages("dplyr")
#remove.packages("tidyverse")
install.packages("tidyverse")
library(tidyverse)
require(tidyverse)


In [ ]:
# make a matrix of all the samples add a column to reflect tissue
p0.05_1.5_DGE_filenames <- list.files("../data", pattern="*0.05_1.5_DGE.csv", all.files=FALSE, full.names=TRUE)
length(p0.05_1.5_DGE_filenames)
# preserve the ordered rownames for later assignment to matrix
t1 <- read.csv(p0.05_1.5_DGE_filenames[1])
t1rownames <- as.character(rownames(t1))
t1$ID <- t1rownames
t2 <- read.csv(p0.05_1.5_DGE_filenames[2])
t2rownames <- as.character(rownames(t2))
t2$ID <- t2rownames
head(t1)
head(t2)


In [ ]:
t1t2 <- dplyr::full_join(t1,t2, by = ("ID"))
head (t1t2)

In [ ]:
# Make a matrix for each of the tissues
# from what files are saved

make_tissue_matrix_ready <- function (file, logFC_mat) {
    filename        <- paste('../data',file,sep="/")
    logFC_mat       <- read.csv(filename)
    logFC_mat       <- logFC_mat[order(rownames(logFC_mat)),]
    logFC           <- as.matrix(as.numeric(logFC_mat$logFC),ncol=1)
    rownames(logFC) <- rownames(logFC_mat)
    return(logFC)
}

In [ ]:

matrix_list <- lapply(                     X  = filenames, 
                                          FUN = make_tissue_matrix_ready, 
                                    logFC_mat = all_se_jc_inc_logFC)
length(matrix_list)


In [ ]:
length(matrix_list)
all_se_jc_inc_logFC_mat = as.matrix(lapply(X   = matrix_list, 
                                           FUN = cbind),
                                          nrow = dim(ordered_merged_rmats)[2], 
                                          ncol = length(matrix_list))
length(all_se_jc_inc_logFC_mat)

In [ ]:
head(all_se_jc_inc_logFC)


In [ ]:
head(tissue_shared_male_female)
tissue <- str_replace(tissue_shared_male_female,'_DGE.txt','')
head(tissue)

In [ ]:
get_tissue_name <- function (tissue_name) {
            tissue <- str_replace(tissue_name,'_DGE.csv','')
            return(tissue)
}

In [ ]:
tissue_list <- lapply(X=levels(tissue_shared_male_female), FUN=get_tissue_name)
length(tissue_list)
head(tissue_list)

Using the matrix_list - create a matrix for calculation of tissue comparisons

In [ ]:
length(unlist(matrix_list[1]))
length(unlist(matrix_list[2]))

In [ ]:
all_se_jc_inc_logFC_mat = as.matrix(as.numeric(unlist(matrix_list[1]), ncol=1))
for (i in (2:length(matrix_list))) {
    n = as.matrix(as.numeric(unlist(matrix_list[i]), ncol=1))
    all_se_jc_inc_logFC_mat = cbind(all_se_jc_inc_logFC_mat, n)
}

dim(all_se_jc_inc_logFC_mat)
rownames(all_se_jc_inc_logFC_mat) = all_se_jc_inc_logFC_mat_rownames
colnames(all_se_jc_inc_logFC_mat) = tissue_list
head(all_se_jc_inc_logFC_mat)

In [ ]:
dim(all_se_jc_inc_logFC_mat)

In [ ]:
length(tissue_list)

With the rmats PSI or se.inc.jc results, are expressed as percentages, there is no need to further further normalize 

In [ ]:
all_se_jc_inc_dist_mat <- as.matrix(cor(all_se_jc_inc_logFC_mat))

In [ ]:
head(all_se_jc_inc_dist_mat)

In [ ]:
rownames(all_se_jc_inc_dist_mat) <- colnames(all_se_jc_inc_logFC_mat)
colnames(all_se_jc_inc_dist_mat) <- colnames(all_se_jc_inc_logFC_mat)

message("Saving all_se_jc_inc_dist_mat object")
saveRDS(object = all_se_jc_inc_dist_mat, file = "../data/all_se_jc_inc_dist_mat.rds")
message("Done!")

In [ ]:
library(pheatmap)
pheatmap(as.matrix(all_se_jc_inc_dist_mat),   fontsize = 6)
hm.parameters <- list(all_se_jc_inc_dist_mat, fontsize = 6)
do.call("pheatmap", c(hm.parameters,  filename="../pdf/Figure1cwithrMATS.all_se_jc_inc_PSI.values.pdf"))

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
figure_id   = "DEFigurewithrMATS"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]